In [1]:
# This script aims to grab frames from video ROV video footage and create jpegs with exif information added

# import required packages
import cv2

# Read the video from specified path this could clearly be changed to a file open box
cap = cv2.VideoCapture("FUWL_FISH_SURVEY_180307_194840_515.asf")

# Loop through specified frames currently I like being able to specify specific start and end points as well as image grab frequency
#but this does need to be changed to be realitve to the timing of the footage and frame rate of footage to make more user friendly
for i in range(1, 2, 1):
    
    # choose specified frame
    cap.set(cv2.CAP_PROP_POS_FRAMES,i)
    # read specified frame
    ret, frame = cap.read()
    # write specified frame to jpg image
    cv2.imwrite('test2'+str(i)+'.jpg',frame)

    # add exif information currenlty listed are the fields that agisoft metashape is expecting

      
# Size(pix)
# Date and time
# Camera make
# Camera model
# Focal length
# F-stop
# Shutter speed
# 35mm focal equivalent
# Sensor x res
# Sensor Y res
# Orientation

# close video file
cap.release()
cv2.destroyAllWindows()


NameError: name 'Image' is not defined

In [26]:

import io
from PIL import Image
import piexif

o = io.BytesIO()
thumb_im = Image.open("test21.jpg")
thumb_im.thumbnail((50, 50), Image.ANTIALIAS)
thumb_im.save(o, "jpeg")
thumbnail = o.getvalue()

zeroth_ifd = {piexif.ImageIFD.Make: u"Canon",
              piexif.ImageIFD.XResolution: (96, 1),
              piexif.ImageIFD.YResolution: (96, 1),
              piexif.ImageIFD.Software: u"piexif"
              }
exif_ifd = {piexif.ExifIFD.DateTimeOriginal: u"2099:09:29 10:10:10",
            piexif.ExifIFD.LensMake: u"LensMake",
            piexif.ExifIFD.Sharpness: 65535,
            piexif.ExifIFD.LensSpecification: ((1, 1), (1, 1), (1, 1), (1, 1)),
            }
gps_ifd = {piexif.GPSIFD.GPSVersionID: (2, 0, 0, 0),
           piexif.GPSIFD.GPSAltitudeRef: 1,
           piexif.GPSIFD.GPSDateStamp: u"1999:99:99 99:99:99",
           }
first_ifd = {piexif.ImageIFD.Make: u"Canon",
             piexif.ImageIFD.XResolution: (40, 1),
             piexif.ImageIFD.YResolution: (40, 1),
             piexif.ImageIFD.Software: u"piexif"
             }

exif_dict = {"0th":zeroth_ifd, "Exif":exif_ifd, "GPS":gps_ifd, "1st":first_ifd, "thumbnail":thumbnail}
exif_bytes = piexif.dump(exif_dict)
im = Image.open("test21.jpg")
im.thumbnail((100, 100), Image.ANTIALIAS)
im.save("out.jpg", exif=exif_bytes)

In [13]:
#  # strip text from image
    
#     # Import required packages
# import cv2
# import pytesseract

# # Mention the installed location of Tesseract-OCR in your system
# pytesseract.pytesseract.tesseract_cmd = 'System_path_to_tesseract.exe'

# # Read image from which text needs to be extracted
# img = cv2.imread("test2101.jpg")

# # Preprocessing the image starts

# # Convert the image to gray scale
# gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# # Performing OTSU threshold
# ret, thresh1 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)

# # Specify structure shape and kernel size.
# # Kernel size increases or decreases the area
# # of the rectangle to be detected.
# # A smaller value like (10, 10) will detect
# # each word instead of a sentence.
# rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (18, 18))

# # Appplying dilation on the threshold image
# dilation = cv2.dilate(thresh1, rect_kernel, iterations = 1)

# # Finding contours
# contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL,
# 												cv2.CHAIN_APPROX_NONE)

# # Creating a copy of image
# im2 = img.copy()

# # A text file is created and flushed
# file = open("recognized.txt", "w+")
# file.write("")
# file.close()

# # Looping through the identified contours
# # Then rectangular part is cropped and passed on
# # to pytesseract for extracting text from it
# # Extracted text is then written into the text file
# for cnt in contours:
# 	x, y, w, h = cv2.boundingRect(cnt)
	
# 	# Drawing a rectangle on copied image
# 	rect = cv2.rectangle(im2, (x, y), (x + w, y + h), (0, 255, 0), 2)
	
# 	# Cropping the text block for giving input to OCR
# 	cropped = im2[y:y + h, x:x + w]
	
# 	# Open the file in append mode
# 	file = open("recognized.txt", "a")
	
# 	# Apply OCR on the cropped image
# 	text = pytesseract.image_to_string(cropped)
	
# 	# Appending the text into file
# 	file.write(text)
# 	file.write("\n")
	
# 	# Close the file
# 	file.close


TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.